改用 [Cimy](https://www.cimy.cc/)

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import yt_dlp
import re
import time
import urllib.parse

# 設定影片目標 URL 和下載選項
episode_url = "https://www.cimy.cc/play/12615-1-1/"
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',
    'merge_output_format': 'mp4',
    'outtmpl': '第一集.%(ext)s'
}

# 設置 Selenium Chrome 瀏覽器
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
service = Service()
driver = webdriver.Chrome(service=service, options=options)

def get_m3u8_url_dynamic(page_url):
    driver.get(page_url)
    time.sleep(3)  # 等待頁面完全加載

    # 解析頁面 HTML 內容
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    m3u8_url = None

    # 檢查每個 iframe 並嘗試取得其內容中的 m3u8 連結
    for iframe in soup.find_all('iframe'):
        if 'src' in iframe.attrs:
            iframe_src = iframe['src']
            # 確保 URL 是完整的（若不是則自動補全）
            if not urllib.parse.urlparse(iframe_src).scheme:
                iframe_src = urllib.parse.urljoin(page_url, iframe_src)

            # 訪問嵌入 iframe 網址
            driver.get(iframe_src)
            time.sleep(2)
            m3u8_match = re.search(r'https?://[^\s\'"]+\.m3u8', driver.page_source)
            if m3u8_match:
                m3u8_url = m3u8_match.group(0)
                break

    return m3u8_url

# 開始解析和下載
m3u8_url = get_m3u8_url_dynamic(episode_url)
driver.quit()  # 關閉瀏覽器

if m3u8_url:
    print(f"找到 .m3u8 連結: {m3u8_url}")
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([m3u8_url])
        print("下載完成！")
    except yt_dlp.utils.DownloadError:
        print("下載失敗，請檢查 .m3u8 連結是否有效。")
else:
    print("未找到有效的 .m3u8 連結。")


找到 .m3u8 連結: https://api.heimuer.app/play/a7df4561c6544227a460109c0f132b19.m3u8
[generic] Extracting URL: https://api.heimuer.app/play/a7df4561c6544227a460109c0f132b19.m3u8
[generic] a7df4561c6544227a460109c0f132b19: Downloading webpage
[generic] a7df4561c6544227a460109c0f132b19: Downloading m3u8 information
[generic] a7df4561c6544227a460109c0f132b19: Checking m3u8 live status
[info] a7df4561c6544227a460109c0f132b19: Downloading 1 format(s): 0
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 955
[download] Destination: 第一集.mp4
[download] 100% of    1.51GiB in 00:09:18 at 2.76MiB/s                   
[FixupM3u8] Fixing MPEG-TS in MP4 container of "第一集.mp4"
下載完成！


自動下載 1 到 20 集的影片

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import yt_dlp
import re
import time
import urllib.parse

# 基本 URL 結構
base_episode_url = "https://www.cimy.cc/play/12615-1-{}"  # URL 格式
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',  # 選擇最高解析度並確保包含音訊
    'merge_output_format': 'mp4',          # 合併為 mp4 格式
}

# 設置 Selenium Chrome 瀏覽器
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
service = Service()
driver = webdriver.Chrome(service=service, options=options)

def get_m3u8_url_dynamic(page_url):
    driver.get(page_url)
    time.sleep(3)  # 等待頁面完全加載

    # 解析頁面 HTML 內容
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    m3u8_url = None

    # 檢查每個 iframe 並嘗試取得其內容中的 m3u8 連結
    for iframe in soup.find_all('iframe'):
        if 'src' in iframe.attrs:
            iframe_src = iframe['src']
            # 確保 URL 是完整的（若不是則自動補全）
            if not urllib.parse.urlparse(iframe_src).scheme:
                iframe_src = urllib.parse.urljoin(page_url, iframe_src)

            # 訪問嵌入 iframe 網址
            driver.get(iframe_src)
            time.sleep(2)
            m3u8_match = re.search(r'https?://[^\s\'"]+\.m3u8', driver.page_source)
            if m3u8_match:
                m3u8_url = m3u8_match.group(0)
                break

    return m3u8_url

# 逐集解析和下載
for episode_number in range(9, 21):
    episode_url = base_episode_url.format(episode_number)
    print(f"正在嘗試下載第 {episode_number} 集: {episode_url}")
    m3u8_url = get_m3u8_url_dynamic(episode_url)
    
    if m3u8_url:
        print(f"第 {episode_number} 集找到 .m3u8 連結: {m3u8_url}")
        output_filename = f"第{episode_number}集.%(ext)s"
        try:
            with yt_dlp.YoutubeDL({**ydl_opts, 'outtmpl': output_filename}) as ydl:
                ydl.download([m3u8_url])
            print(f"第 {episode_number} 集下載完成！")
        except yt_dlp.utils.DownloadError:
            print(f"第 {episode_number} 集下載失敗，請檢查 .m3u8 連結是否有效。")
    else:
        print(f"第 {episode_number} 集未找到有效的 .m3u8 連結。")

# 關閉瀏覽器
driver.quit()


正在嘗試下載第 9 集: https://www.cimy.cc/play/12615-1-9
第 9 集找到 .m3u8 連結: https://api.heimuer.app/play/6ec1d3db23e74f3cb4fe3714005ec1fc.m3u8
[generic] Extracting URL: https://api.heimuer.app/play/6ec1d3db23e74f3cb4fe3714005ec1fc.m3u8
[generic] 6ec1d3db23e74f3cb4fe3714005ec1fc: Downloading webpage
[generic] 6ec1d3db23e74f3cb4fe3714005ec1fc: Downloading m3u8 information
[generic] 6ec1d3db23e74f3cb4fe3714005ec1fc: Checking m3u8 live status
[info] 6ec1d3db23e74f3cb4fe3714005ec1fc: Downloading 1 format(s): 0
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 959
[download] Destination: 第9集.mp4
[download] 100% of    1.52GiB in 00:04:22 at 5.93MiB/s                   
[FixupM3u8] Fixing MPEG-TS in MP4 container of "第9集.mp4"
第 9 集下載完成！
正在嘗試下載第 10 集: https://www.cimy.cc/play/12615-1-10
第 10 集找到 .m3u8 連結: https://api.heimuer.app/play/613e19d4434d4b3fb9a856c4eb7bd95b.m3u8
[generic] Extracting URL: https://api.heimuer.app/play/613e19d4434d4b3fb9a856c4eb7bd95b.m3u8
[generic] 613e19d4434d4b3

[download] Got error: HTTP Error 526: None


[download] fragment not found; Skipping fragment 1 ...


[download] Got error: HTTP Error 526: None


[download] fragment not found; Skipping fragment 2 ...


...

[download] fragment not found; Skipping fragment 244 ...


[download] Got error: HTTP Error 526: None


[download] fragment not found; Skipping fragment 245 ...
[download] fragment not found; Skipping fragment 246 ...
[download] fragment not found; Skipping fragment 247 ...
[download] fragment not found; Skipping fragment 248 ...
[download] fragment not found; Skipping fragment 249 ...
[download] fragment not found; Skipping fragment 250 ...
[download] fragment not found; Skipping fragment 251 ...
[download] fragment not found; Skipping fragment 252 ...
[download] fragment not found; Skipping fragment 253 ...
[download] fragment not found; Skipping fragment 254 ...
[download] fragment not found; Skipping fragment 255 ...
[download] fragment not found; Skipping fragment 256 ...
[download] fragment not found; Skipping fragment 257 ...
[download] fragment not found; Skipping fragment 258 ...
[download] fragment not found; Skipping fragment 259 ...
[download] fragment not found; Skipping fragment 260 ...
[download] fragment not found; Skipping fragment 261 ...
[download] fragment not found; 

[download] Got error: HTTPSConnectionPool(host='4t74723ents.com', port=443): Read timed out. (read timeout=20.0)


[download] fragment not found; Skipping fragment 337 ...
[download] 100% of    1.53GiB in 00:10:35 at 2.47MiB/s                     
[FixupM3u8] Fixing MPEG-TS in MP4 container of "第19集.mp4"
第 19 集下載完成！
正在嘗試下載第 20 集: https://www.cimy.cc/play/12615-1-20
第 20 集找到 .m3u8 連結: https://api.heimuer.app/play/ff49cbfc7be34a71b7e1b0e17cfd53b6.m3u8
[generic] Extracting URL: https://api.heimuer.app/play/ff49cbfc7be34a71b7e1b0e17cfd53b6.m3u8
[generic] ff49cbfc7be34a71b7e1b0e17cfd53b6: Downloading webpage
[generic] ff49cbfc7be34a71b7e1b0e17cfd53b6: Downloading m3u8 information
[generic] ff49cbfc7be34a71b7e1b0e17cfd53b6: Checking m3u8 live status
[info] ff49cbfc7be34a71b7e1b0e17cfd53b6: Downloading 1 format(s): 0
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 994
[download] Destination: 第20集.mp4
[download] 100% of    1.58GiB in 00:10:07 at 2.66MiB/s                    
[FixupM3u8] Fixing MPEG-TS in MP4 container of "第20集.mp4"
第 20 集下載完成！
